<a href="https://colab.research.google.com/github/radanim/HM-AIRS-PROGRAM/blob/master/Transfer_Learning_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.0.0.alpha0 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow-gpu==2.0.0.alpha0


In [2]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
     -O ./cats_and_dogs_filtered.zip

--2022-12-09 07:10:14--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 74.125.24.128, 172.217.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  20.1MB/s    in 3.9s    

2022-12-09 07:10:19 (16.7 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [4]:
import os # 폴더를 다루고 파일을 찾는 경로 작업에 사용 
import zipfile # zip형식 데이터 추출 파이썬 라이브러리 
import numpy as np # 행렬이나 다양한 함수에 사용 
import tensorflow as tf # 신경망 모델링 라이브러리
import matplotlib.pyplot as plt # 이미지 시각화 

from tqdm import tqdm_notebook #프로젝트 진행상황 시각화 
from tensorflow.keras.preprocessing.image import ImageDataGenerator #내부 이미지 전처리 파이프라인 생성

%matplotlib inline
tf.__version__ 

'2.9.2'

In [5]:
dataset_path="./cats_and_dogs_filtered.zip"

In [6]:
zip_object=zipfile.ZipFile(file=dataset_path, mode="r")

In [7]:
zip_object.extractall("./") #./ 현재 디렉토리

In [8]:
zip_object.close()

In [9]:
dataset_path_new="./cats_and_dogs_filtered/"

In [10]:
train_dir=os.path.join(dataset_path_new, "train")
validation_dir=os.path.join(dataset_path_new,"validation")

In [11]:
# 모델 만들기
# 사전학습 모델 로드하기 (MobileNetV2)

In [12]:
IMG_SHAPE=(128,128,3)

In [13]:
base_model=tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9406464/9406464 [==============================] - 0s 0us/step


In [14]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [15]:
# 베이스 모델 동결하기 

In [22]:
base_model.trainable=False

In [ ]:
# 커스텀 헤드 정의하기 

In [17]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [18]:
global_average_layer=tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [19]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [20]:
prediction_layer=tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [20]:
# 모델 정의하기

In [23]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

In [20]:
# 모델 compiling
# 커스텀 헤드 + 사전학습 모델(MobileNetV2)= 커스텀 모델 생성 

In [26]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [20]:
# 이미지 데이터 생성기 만들기 (데이터 증강) 

In [27]:
data_gen_train=ImageDataGenerator(rescale=1/255.)
data_gen_valid=ImageDataGenerator(rescale=1/255.)

In [28]:
train_generator=data_gen_train.flow_from_directory(train_dir,target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [29]:
valid_generator=data_gen_valid.flow_from_directory(validation_dir,target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [20]:
# 모델 훈련   

In [32]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

<ipython-input-32-f08455725960>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)


Epoch 1/5
16/16 [==============================] - 8s 522ms/step - loss: 0.1894 - accuracy: 0.9400 - val_loss: 0.1841 - val_accuracy: 0.9460
Epoch 2/5
16/16 [==============================] - 8s 493ms/step - loss: 0.1791 - accuracy: 0.9440 - val_loss: 0.1748 - val_accuracy: 0.9480
Epoch 3/5
16/16 [==============================] - 8s 491ms/step - loss: 0.1700 - accuracy: 0.9490 - val_loss: 0.1664 - val_accuracy: 0.9510
Epoch 4/5
16/16 [==============================] - 8s 497ms/step - loss: 0.1617 - accuracy: 0.9515 - val_loss: 0.1592 - val_accuracy: 0.9550
Epoch 5/5
16/16 [==============================] - 8s 500ms/step - loss: 0.1544 - accuracy: 0.9530 - val_loss: 0.1527 - val_accuracy: 0.9570


In [20]:
 # 전이학습 모델 평가

In [33]:
valid_loss, valid_accuracy= model.evaluate_generator(valid_generator)

<ipython-input-33-abdb002c7a3b>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy= model.evaluate_generator(valid_generator)


In [34]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9570000171661377


In [20]:
# Fine Tuning (미세 조정)
# 기초 네트워크에서 상위층 몇개의 동결을 해제
# 커스텀 데이터 세트에 대해 네트워크가 더 많이 학습하도록 조절(일부만, 전체 네트워크X)
# CNN 학습을 예로 들면 하위 레이어는 일반적인 특성만 학습(선, 가장자리, 모서리) + 상위레이어(데이터세트의 추상적인 특성 학습)
# 네트워크의 특정부분을 커스텀 데이터 세트에 적용하기 
# 상위 몇개층만 동결해제하기 (기초모델 전체를 동결 후에 진행)

In [35]:
base_model.trainable=True  

In [37]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 154


In [38]:
fine_tune_at=100

In [39]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable=False

In [20]:
# 일부 층이 동결 해제된 모델을 컴파일링

In [40]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [35]:
# 미세조정(Fine Tuning)

In [41]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

<ipython-input-41-f08455725960>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)


Epoch 1/5
16/16 [==============================] - 14s 564ms/step - loss: 0.1716 - accuracy: 0.9320 - val_loss: 0.0697 - val_accuracy: 0.9700
Epoch 2/5
16/16 [==============================] - 8s 506ms/step - loss: 0.0248 - accuracy: 0.9980 - val_loss: 0.0737 - val_accuracy: 0.9760
Epoch 3/5
16/16 [==============================] - 8s 513ms/step - loss: 0.0131 - accuracy: 0.9985 - val_loss: 0.1074 - val_accuracy: 0.9720
Epoch 4/5
16/16 [==============================] - 9s 576ms/step - loss: 0.0066 - accuracy: 0.9995 - val_loss: 0.1612 - val_accuracy: 0.9610
Epoch 5/5
16/16 [==============================] - 8s 511ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.1465 - val_accuracy: 0.9660


In [35]:
# 미세조정을 해도 정확도는 올라갈수도 있고, 내려갈수도 있음. 추가적으로 활용가능한 방법중 하나일뿐. 
# 미세조정 모델 평가하기

In [42]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

<ipython-input-42-a86cc1f7f1e2>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


In [43]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9660000205039978


In [35]:
# 결과는 수행하는 과제나 모델 데이터세트에 따라 다 다름. 